In [1]:
#再帰的に構造ー物性相関データセットから理由を生成させるNotebook
#バッチ処理

#ライブラリの自動インポート
%reload_ext autoreload
%autoreload 2

In [2]:
import openai
from tqdm import tqdm
import pandas as pd
import os
import glob
import json
import copy

In [3]:
#環境構築からapi keyを読み込み
openai.api_key =os.environ["OPENAI_API_KEY"]

In [4]:
#laod dataset as dict
csv_path="dataset/BradleyMeltingPointDataset_clean.csv"
df=pd.read_csv(csv_path)
chemical_records=df.to_dict(orient='records')
chemical_records[:1]

[{'mpC': 87.0,
  'name': '((5-((Diphenylphosphino)methyl)-2,2-dimethyl-1,3-dioxolan-4-yl)methyl)(diphenyl)phosphine',
  'smiles': 'P(CC1OC(OC1CP(c1ccccc1)c1ccccc1)(C)C)(c1ccccc1)c1ccccc1',
  'csid': 109291,
  'link': 'http://dx.doi.org/10.1021/ci0500132',
  'source': 'Karthikeyan M.; Glen R.C.; Bender A. General melting point prediction based on a diverse compound dataset and artificial neural networks. J. Chem. Inf. Model.; 2005; 45(3); 581-4277'}]

In [5]:
model="gpt-4-1106-preview"

In [6]:
system_prompt="""
Provide the quantitative Reason and Prediction so that a scientist, who does not know the melting point, can predict the value.

#Commands
- You must quantitatively consider how the melting point shifts, focusing on each functional groups.
- Actual value and Prediction must match each other.
- If Actual value and Prediction differ each other, rethink Reason.
- If Prediction does not contain numbers for each functional group effect, rethink Reason

#Example reason
- Target compound: Toluene
- Basic unit, benzene has a boiling point of 80.
- Methyl group: +30 (due to larger molecular weight)
- Prediction: 110

"""

In [7]:
def gen_prompt(chemical_record,reason="",prediction=""):
    name=chemical_record["name"]
    smiles=chemical_record["smiles"]
    value=chemical_record["mpC"]
    prompt=f"""
#Data
-Name: {name}
-SMILES: {smiles} 
-Actual value: {value}
-Reason: {reason}
-Prediction: {prediction}

#Output (JSON keys)
- Reason, Prediction
"""
    return prompt


In [8]:
import json

#ask gpt
def json_generate(prompt,model="gpt-3.5-turbo-1106"):
    response = openai.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {
            "role": "user",
            "content": f"""{prompt}"""
        }  
    ],
    response_format={ "type": "json_object" }
    )

    return (json.loads(response.choices[0].message.content))


#parse prediction
def prediction_string_to_number(prompt,model="gpt-3.5-turbo-1106"):
    response = openai.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": """Extract integer from prediction. Use average if multiple numbers are included.
            Examples:
            In: 70.2 - 75.2 degrees Celsius
            Out: 73
            In: 75.2 degrees Celsius
            Out: 73
            In: For 1-naphthalenecarboxaldehyde, starting with the base value for naphthalene with a melting point of 80\u00b0C and subtracting the estimated aldehyde effect of approximately -47 to -50\u00b0C, the predicted melting point would be in the range of 30-33\u00b0C.
            Out: 32
            """,
        },
        {
            "role": "user",
            "content": f"""{prompt}
"#Output (JSON keys)
- Prediction"""
        }  
    ],
    response_format={ "type": "json_object" }
    )

    return (json.loads(response.choices[0].message.content))

In [10]:

save_base_path="dataset/231225AutoReasoning/"


#load finished records
gen_records={}
gen_json_path_list=glob.glob(save_base_path+"*.json")
for gen_json_path in tqdm(gen_json_path_list):
    with open(gen_json_path) as f:
        gen_hist=json.load(f)
    gen_records[gen_hist[0]["name"]]=gen_hist

100%|██████████| 224/224 [00:00<00:00, 36127.21it/s]


In [11]:
import re

def remove_non_alphabet_characters(s):
    # Using regex to remove all non-alphabet characters
    return re.sub('[^a-zA-Z]', '', s)

In [12]:
n_recursion=2
n_random_repeat=3
error_threshold=10

In [13]:
#batch 
for chemical_record in tqdm(chemical_records):

    #load record
    gen_record=copy.deepcopy(chemical_record)

    #skip if already generated
    if gen_record["name"] in gen_records:
        print(f"Skip because already generated: {gen_record['name']}")
        continue

    record_history=[]

    fin_flag=False
    #make suggestion with random seed
    for j in range(n_random_repeat):
        if fin_flag:
            break

        gen_record["Reason"]=""
        gen_record["Prediction"]=""
        if j==0:
            record_history.append(copy.deepcopy(gen_record))

        #improve reasoing
        for i in range(n_recursion):
            try:
                r=json_generate(
                    gen_prompt(gen_record,
                            reason=gen_record["Reason"],
                            prediction=gen_record["Prediction"]
                    ),
                    model=model,
                )
            except Exception as e:
                #error occurs especially when generatin JSON data by gpt
                print(e)
                fin_flag=True
                break
            #parse prediction string to number
            gen_record.update(r)
            try:
                gen_record["Prediction(integer)"]=float(prediction_string_to_number(gen_record["Prediction"])["Prediction"])
            except:
                gen_record["Prediction(integer)"]=99999
            record_history.append(copy.deepcopy(gen_record))
            
            #finish reasoning if prediction is close to actual value
            if abs(gen_record["Prediction(integer)"]-gen_record["mpC"])<=error_threshold:
                fin_flag=True
                print(f"Finished because good reasoning was achieved: {gen_record['name']}")
                break

    #save
    save_name=remove_non_alphabet_characters(gen_record["name"])
    save_path=save_base_path+f"{save_name}.json"
    with open(save_path, 'w') as f:
        json.dump(record_history, f, indent=4)

    gen_records[gen_record["name"]]=record_history

  0%|          | 0/24889 [00:00<?, ?it/s]

Skip because already generated: ((5-((Diphenylphosphino)methyl)-2,2-dimethyl-1,3-dioxolan-4-yl)methyl)(diphenyl)phosphine
Skip because already generated: (+)-alpha-pinene
Skip because already generated: (+)-camphene
Skip because already generated: (+)-fenchol
Skip because already generated: (+)-isomenthol
Skip because already generated: (+)-perseitol
Skip because already generated: (+-)camphor
Skip because already generated: (+/+)-ibuprofen
Skip because already generated: (+/+)-indoprofen
Skip because already generated: (+/+)-ketoprofen
Skip because already generated: (-)-2-amino-3-phenyl-1-propanol
Skip because already generated: (-)-beta-caryophyllene epoxide
Skip because already generated: (-)-levonorgestrel
Skip because already generated: (-)-menthone
Skip because already generated: (1'S,3aR,4R,6'S,7aR,9'R,10'S)-7a,10'-Dimethyltetrahydro-1H,11'H-spiro[2-benzofuran-4,5'-[3]oxatricyclo[7.2.1.0~1,6~]dodecane]-2',3,11'(3aH)-trione
Skip because already generated: (1,1'-binaphthalene)-2,

  0%|          | 29/24889 [00:32<7:49:20,  1.13s/it]

Finished because good reasoning was achieved: (1-bromoethyl)benzene
Skip because already generated: (1-methyl-1H-benzimidazol-2-yl)(3-nitropyridin-2-yl)acetonitrile
Skip because already generated: (10S)-10-Hydroxy-10-(2-oxopropyl)phenanthren-9(10H)-one
Skip because already generated: (11beta)-11,17,21-Trihydroxypregn-4-ene-3,20-dione
Skip because already generated: (11beta)-11,17-Dihydroxy-3,20-dioxopregna-1,4-dien-21-yl acetate
Skip because already generated: (13E,14aR,18aR,19E)-5,8,14a,15,16,17,18,18a-octahydrotribenzo[b,f,l][1,4,8,11]tetraazacyclotetradecine-6,7-dione
Skip because already generated: (13Z)-Docos-13-enamide
Skip because already generated: (13aS)-10-Hydroxy-9-methoxy-5,6,13,13a-tetrahydro-8H-[1,3]dioxolo[4,5-g]isoquinolino[3,2-a]isoquinolin-8-one
Skip because already generated: (13cR)-13c-Methoxy-3,13c-dihydrodibenzo[a,kl]xanthen-1(2H)-one
Skip because already generated: (17-oxoandrost-2-en-3-yl)acetic acid
Skip because already generated: (17beta)-17-Ethynyl-3,17-dimet

  0%|          | 46/24889 [00:43<6:16:13,  1.10it/s]

...

Finished because good reasoning was achieved: (2E)-4,4,4-trifluoro-2-{[5-(4-methylphenyl)-1H-pyrazol-3-yl]hydrazono}-1-thiophen-2-ylbutane-1,3-dione


  2%|▏         | 401/24889 [2:27:32<458:48:11, 67.45s/it]

Finished because good reasoning was achieved: (2E)-4-[5-chloro-3-(5-methylfuran-2-yl)-1-benzofuran-2-yl]butan-2-one oxime


  2%|▏         | 402/24889 [2:27:46<348:10:10, 51.19s/it]

Finished because good reasoning was achieved: (2E)-4-methyl-1-phenylpentan-2-one semicarbazone
